In [1]:
import pathlib
import shutil

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
pathlib.Path().joinpath('..', 'data', 'images', 'train').mkdir(exist_ok=True)
pathlib.Path().joinpath('..', 'data', 'images', 'test').mkdir(exist_ok=True)

In [3]:
image_info_df = pd.read_csv(pathlib.Path().joinpath('..', 'data', 'informations', 'train_labels.csv'))
image_info_df

,id,label
0,f38a6374c348f90b587e046aac6079959adf3835,0
1,c18f2d887b7ae4f6742ee445113fa1aef383ed77,1
2,755db6279dae599ebb4d39a9123cce439965282d,0
3,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08,0
4,068aba587a4950175d04c680d38943fd488d6a9d,0
...,...,...
220020,53e9aa9d46e720bf3c6a7528d1fca3ba6e2e49f6,0
220021,d4b854fe38b07fe2831ad73892b3cec877689576,1
220022,3d046cead1a2a5cbe00b2b4847cfb7ba7cf5fe75,0
220023,f129691c13433f66e1e0671ff1fe80944816f5a2,0


In [4]:
filenames = [filename.name.split('.')[0] for filename in
             pathlib.Path().joinpath("..", "data", "images", "raw", "train").iterdir()]
filenames_df = pd.DataFrame(data={'id': filenames})
filenames_df = filenames_df.sample(n=10000)

In [5]:
imgs_df = filenames_df.merge(image_info_df, how='inner', on='id')

In [6]:
imgs_df['set'] = np.random.choice(['train', 'test'], imgs_df.shape[0], p=[0.7, 0.3])
imgs_df

,id,label,set
0,d03d75f2785286510118020dec96eb112dddd924,0,train
1,f9c86a26e36f103a3e46ca2a171b1f2ce84c5e7b,1,train
2,0e6622c9f9e885adbba4a301db99b5cefc4fad2b,1,train
3,42165e58034c1d1f5aaea90a25f1b8960b439eb0,0,test
4,e67763ab96956c992efc39f4e2437979fd80c491,1,test
...,...,...,...
9995,6effdd0cb4867fe5ae703fba16a9d768b50d4fb2,0,train
9996,468b01d7c968f7eb156157f06e178e1a2ebf2fdc,0,test
9997,cf36e3fc2d5f7151aa25c671e5ae50b12bc3a338,0,test
9998,5921d6e4de70bca65cafb124237ee5bcd3ea90ac,1,train


In [7]:
def move_data_into_folder(df: pd.DataFrame, src: pathlib.Path, dst: pathlib.Path, ext='.tif'):
    for idx, row in tqdm(df.iterrows(), total=df.shape[0], desc='Data transfer'):
        id = row['id']
        set = row['set']
        id = ''.join([id, ext])
        if set == 'train':
            shutil.move(src=src.joinpath(id), dst=dst.joinpath(set, id))
        else:
            shutil.move(src=src.joinpath(id), dst=dst.joinpath(set, id))



In [8]:
move_data_into_folder(df = imgs_df, src = pathlib.Path('..', 'data', 'images', 'raw', 'train'),
                      dst=pathlib.Path('..', 'data', 'images'))

Data transfer:   0%|          | 0/10000 [00:00<?, ?it/s]

In [9]:
imgs_df.to_csv(pathlib.Path('..', 'data', 'informations', 'sample.csv'), index=False)